### Tests for compiler and knowledge base

In [4]:
import sys
import os

openagent_dir = os.path.dirname(os.path.abspath(""))
sys.path.append(openagent_dir)

In [12]:
from openagent import compiler
from openagent.llms._openai import OpenAI
from openagent.vectorstores.chroma import Chroma
from openagent.text_splitter import CharacterTextSplitter
from openagent.knowledgebase.document_loaders.file.base import SimpleDirectoryReader
from openagent.vectorstores.embeddings.openai import OpenAIEmbeddings

# load .env file in format :
# OPENAI_API_KEY=sk-xxx
from dotenv import load_dotenv
load_dotenv()


llmcc = OpenAI(
    model="gpt-3.5-turbo"
)

llmtc = OpenAI(
    model="text-davinci-003"
)

#### Test case for known data (already in llm context)

In [19]:
pg_data = 'test_data'
loader = SimpleDirectoryReader(input_dir=pg_data)
docs = loader.load_data()

splitter = CharacterTextSplitter(chunk_overlap=400, chunk_size=2000)
splitted = splitter.split_documents(docs)

openai_embeddings = OpenAIEmbeddings()
chroma = Chroma(embedding_function=openai_embeddings)

chroma.add_documents(splitted)

[Document(id_='46aeea1e-2382-4f9e-9b69-6d54a22daf7d', embedding=None, metadata={'page_label': '1', 'file_name': 'ml-ops.pdf', 'source': 'test_data'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='eb46eceba4d4afe762a36784b2f42c6bd23300c1fd6c69e15043bb7ae1ee24da', text=' MLOps: A Step Forward to Enterpr ise Machine \nLearning  \nAbdullah Ikram Ullah Tabassam  \nDepartment of Computer Science and \nMathematics  \nLiverpool J ohn Moores University  \nLiverpool, United Kingdom  \nabdullahdar2017@gmail.com  \ncsmaulla@ljmu.ac.uk\nAbstract— Machine Learning Operations (MLOps) is \nbecoming a highly crucial part of businesses  looking to \ncapitalize  on the benefits of AI and ML models. This \nresearch presents a detailed review of MLOps, its \nbenefits, difficulties, evolutions, and important \nunderly ing technologies such as MLOps frameworks, \nDocker, GitHub actions , and Kubernetes. The MLOps \nworkflow, which includes model design, deployment, a

['8cc3794a-3674-11ee-b60b-88b111a71c81',
 '8cc399e4-3674-11ee-98a7-88b111a71c81',
 '8cc399e5-3674-11ee-b082-88b111a71c81',
 '8cc399e6-3674-11ee-b8f9-88b111a71c81',
 '8cc399e7-3674-11ee-9aea-88b111a71c81',
 '8cc399e8-3674-11ee-8297-88b111a71c81',
 '8cc399e9-3674-11ee-9092-88b111a71c81',
 '8cc399ea-3674-11ee-bdca-88b111a71c81',
 '8cc399eb-3674-11ee-a0b8-88b111a71c81',
 '8cc399ec-3674-11ee-9f19-88b111a71c81',
 '8cc399ed-3674-11ee-99ea-88b111a71c81',
 '8cc399ee-3674-11ee-bac4-88b111a71c81',
 '8cc399ef-3674-11ee-b229-88b111a71c81',
 '8cc399f0-3674-11ee-b51c-88b111a71c81',
 '8cc399f1-3674-11ee-9369-88b111a71c81',
 '8cc399f2-3674-11ee-bdf0-88b111a71c81',
 '8cc399f3-3674-11ee-9e80-88b111a71c81',
 '8cc399f4-3674-11ee-96ea-88b111a71c81',
 '8cc399f5-3674-11ee-a556-88b111a71c81',
 '8cc399f6-3674-11ee-b910-88b111a71c81',
 '8cc399f7-3674-11ee-ba1e-88b111a71c81',
 '8cc399f8-3674-11ee-8dfa-88b111a71c81',
 '8cc399f9-3674-11ee-a04c-88b111a71c81',
 '8cc399fa-3674-11ee-88e9-88b111a71c81',
 '8cc399fb-3674-

In [20]:
def knowledgebase(query):
    doc = chroma.similarity_search(query=query, top_k=1)
    return doc[0][0].page_content

In [21]:
knowledgebase('we had enough money')

"Another thing I didn't get at the time is that growth rate is the ultimate test of a startup. Our growth rate was fine. We had about 70 stores at the end of 1996 and about 500 at the end of 1997. I mistakenly thought the thing that mattered was the absolute number of users. And that is the thing that matters in the sense that that's how much money you're making, and if you're not making enough, you might go out of business. But in the long term the growth rate takes care of the absolute number. If we'd been a startup I was advising at Y Combinator, I would have said: Stop being so stressed out, because you're doing fine. You're growing 7x a year. Just don't hire too many more people and you'll soon be profitable, and then you'll control your own destiny.\n\nAlas I hired lots more people, partly because our investors wanted me to, and partly because that's what startups did during the Internet Bubble. A company with just a handful of employees would have seemed amateurish. So we didn't

In [22]:
# chatbot demonstration

def chatbot(input, chat_loop):

    knowledge = knowledgebase(input)
    if chat_loop is None:
        chat_loop = compiler(
                    '''{{#system}}You are a helpful assistant. {{/system}}

                        {{~#geneach 'conversation' stop=False}}

                        {{#user~}}
                        Query: {{set 'this.user_text' (await 'user_text') hidden=False}}
                        Use this below information for your answer. Do not talk about the information provided, Just answer the query as an intelligent Assistant
                        {{knowledge}}
                        {{~/user}}

                        {{#assistant~}}
                        {{gen 'this.response' temperature=0 max_tokens=300}}
                        {{~/assistant}}
                        
                        {{~/geneach}}''', llm = llmcc, silent = True, caching = False) 
    
    chat_loop = chat_loop(user_text = input, knowledge = knowledge) 
    return chat_loop['conversation'][-2]['response'], chat_loop

In [23]:
resp, loop = chatbot("If I don't hire too many more people and will I become profitable?", None)
print(resp)

Based on the information provided, if you don't hire too many more people, you have a good chance of becoming profitable. The growth rate of your startup is strong, growing 7x a year, which indicates positive potential for financial success. However, hiring too many additional employees may hinder your profitability. It is important to control your expenses and focus on sustainable growth to ensure long-term success and control over your company's destiny.


In [24]:
resp, loop = chatbot("How to get registered in YC?", loop)
print(resp)

To get registered in Y Combinator (YC), you will need to go through their application process. YC is a startup accelerator that selects and invests in early-stage companies. Here are the general steps to apply:

1. Visit the YC website: Go to the YC website and navigate to the application page.

2. Fill out the application: Complete the application form, providing information about your startup, team, market, and other relevant details. Be thorough and concise in your responses.

3. Submit your application: Once you have filled out the application, submit it through the designated submission process on the YC website. Make sure to review your application for any errors or missing information before submitting.

4. Wait for a response: YC receives a large number of applications, so it may take some time for them to review and respond to your application. Be patient and wait for their decision.

5. Interview: If your application is shortlisted, you may be invited for an interview with th

#### Test case for hypothetical data (not in llm context)

In [ ]:
pg_data = 'test_data'
loader = SimpleDirectoryReader(input_dir=pg_data)
docs = loader.load_data()

splitter = CharacterTextSplitter(chunk_overlap=20, chunk_size=1024)
splitted = splitter.split_documents(docs)

openai_embeddings = OpenAIEmbeddings()
chroma = Chroma(embedding_function=openai_embeddings)

chroma.add_documents(splitted)

In [78]:
chroma.similarity_search(query="What did Alex said to Xarnak in their first meeting?", top_k=1)

[(Document(page_content="ALEX:\n(Defiant)\nI am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it.\n\n(The two adversaries lock eyes, their powers crackling around them.)\n\nNARRATOR (V.O.):\nThe fate of Earth hung in the balance as Alex and Xarnak prepared to engage in an epic battle that would decide the planet's destiny.\n\n(With a mighty roar, Xarnak unleashes a massive energy blast toward Alex. In a display of agility, Alex dodges the attack with astonishing precision.)\n\nALEX:\n(Focused)\nYou can't defeat me, Xarnak. I've become more than you ever imagined.\n\n(Xarnak's face contorts with rage, and he charges at Alex, unleashing a barrage of powerful strikes. But Alex's speed proves too much for the alien warlord, who struggles to keep up.)\n\nNARRATOR (V.O.):\nThe Meteoric Guardian's super-speed gave him the upper hand in the battle, as he evaded Xarnak's attacks and countered with remarkable accuracy.", metadata={'s

In [79]:
def knowledgebase(query):
    doc = chroma.similarity_search(query=query, top_k=1)
    return doc[0][0].page_content

In [80]:
# chatbot demonstration

def chatbot(input, chat_loop):

    knowledge = knowledgebase(input)
    if chat_loop is None:
        chat_loop = compiler(
                    '''{{#system}}You are a helpful assistant. {{/system}}

                        {{~#geneach 'conversation' stop=False}}

                        {{#user~}}
                        Query: {{set 'this.user_text' (await 'user_text') hidden=False}}
                        Use this below information for your answer. Do not talk about the information provided, Just answer the query as an intelligent Assistant
                        {{knowledge}}
                        {{~/user}}

                        {{#assistant~}}
                        {{gen 'this.response' temperature=0 max_tokens=300}}
                        {{~/assistant}}
                        
                        {{~/geneach}}''', llm = llmcc, silent = True, caching = False) 
    
    chat_loop = chat_loop(user_text = input, knowledge = knowledge) 
    return chat_loop['conversation'][-2]['response'], chat_loop

In [81]:
resp, loop = chatbot("What did Alex said to Xarnak in their first meeting?", None)
print(resp)

I am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it. You can't defeat me, Xarnak. I've become more than you ever imagined.


#### Using llama-index

In [25]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('test_data\meteoric').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What did Alex said to Xarnak in their first meeting?")
print(response)


ALEX: "I am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it."
